In [1]:
!pip install -q kaggle
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ifeanyinneji","key":"68b7b1b6c0d102f7d9941d0b3eff800a"}'}

In [2]:
# This cell creates a .kaggle hidden folder in our root directory and copies the kaggle.json file to the folder.
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/
! chmod 600 /root/.kaggle/kaggle.json

In [5]:
# This cell will download the dataset used for this project to the /content directory and also unzip the csv file.
!kaggle datasets download -d deepcontractor/marvel-comic-books
!unzip /content/marvel-comic-books.zip

  0% 0.00/3.58M [00:00<?, ?B/s]
100% 3.58M/3.58M [00:00<00:00, 186MB/s]
Archive:  /content/marvel-comic-books.zip
  inflating: Marvel_Comics.csv       


In [6]:
!du -sh *

3.5M	Apr28_2021.csv
5.7M	Aug2020.csv
4.0K	kaggle.json
4.9M	manga-dataset.zip
3.6M	marvel-comic-books.zip
13M	Marvel_Comics.csv
3.5M	May2_2021.csv
7.0M	May3_2021.csv
55M	sample_data


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('darkgrid')
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
!pip install fastparquet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 19.8 MB/s 
     |████████████████████████████████| 1.6 MB 41.1 MB/s 
     |████████████████████████████████| 140 kB 46.6 MB/s 


In [3]:
import pandas as pd
df = pd.read_csv("Marvel_Comics.csv")
df.to_parquet("comics.parquet")
df = pd.read_parquet("comics.parquet", engine="fastparquet")
print("Shape", df.shape)


Shape (34992, 12)


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34992 entries, 0 to 34991
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   comic_name         34992 non-null  object
 1   active_years       34992 non-null  object
 2   issue_title        34992 non-null  object
 3   publish_date       34992 non-null  object
 4   issue_description  34992 non-null  object
 5   penciler           34992 non-null  object
 6   writer             34992 non-null  object
 7   cover_artist       34992 non-null  object
 8   Imprint            34992 non-null  object
 9   Format             34992 non-null  object
 10  Rating             34992 non-null  object
 11  Price              34992 non-null  object
dtypes: object(12)
memory usage: 3.2+ MB


In [5]:
df.head()

,comic_name,active_years,issue_title,publish_date,issue_description,penciler,writer,cover_artist,Imprint,Format,Rating,Price
0,A Year of Marvels: April Infinite Comic (2016),(2016),A Year of Marvels: April Infinite Comic (2016) #1,"April 01, 2016",The Infinite Comic that will have everyone tal...,Yves Bigerel,Yves Bigerel,Jamal Campbell,Marvel Universe,Infinite Comic,Rated T+,Free
1,A Year of Marvels: August Infinite Comic (2016),(2016),A Year of Marvels: August Infinite Comic (2016...,"August 10, 2016","It’s August, and Nick Fury is just in time to ...",Jamal Campbell,"Chris Sims, Chad Bowers",None,Marvel Universe,Infinite Comic,None,Free
2,A Year of Marvels: February Infinite Comic (2016),(2016),A Year of Marvels: February Infinite Comic (20...,"February 10, 2016",Join us in a brand new Marvel comics adventure...,"Danilo S. Beyruth, M Mast",Ryan North,None,Marvel Universe,Infinite Comic,Rated T+,Free
3,A Year of Marvels: July Infinite Comic (2016),(2016),A Year of Marvels: July Infinite Comic (2016) #1,"June 29, 2016",Celebrating the Fourth of July is complicated ...,Juanan Ramirez,Chuck Wendig,Jamal Campbell,Marvel Universe,Infinite Comic,None,Free
4,A Year of Marvels: June Infinite Comic (2016),(2016),A Year of Marvels: June Infinite Comic (2016) #1,"June 15, 2016",Sam Alexander’s finding it hard to cope with t...,Diego Olortegui,Paul Allor,Jamal Campbell,Marvel Universe,Infinite Comic,None,Free


In [6]:
total_genres = []
for g in df["Rating"]:
    g = g
    if g not in total_genres:
        total_genres.append(g)
total_genres

[' Rated T+',
 'None',
 ' Rated T',
 ' ALL AGES',
 ' A',
 ' Parental Advisory',
 ' Marvel Psr',
 ' No Rating',
 ' MARVEL PSR',
 ' T',
 ' RATED T',
 ' Max',
 ' RATED T+',
 ' RATED A',
 ' All Ages',
 ' T+',
 ' Rated a',
 ' Rated A',
 ' Parental Advisory/Explicit Content',
 ' PARENTAL SUPERVISION',
 ' PARENTAL ADVISORY',
 ' Mature',
 ' MARVEL PSR+',
 ' EXPLICIT CONTENT',
 ' PARENTAL ADVISORYSLC',
 ' Parental AdvisorySLC',
 ' Parental Advisoryslc',
 ' Explicit Content',
 ' PARENTAL ADVISORY/EXPLICIT CONTENT',
 ' NO RATING',
 ' NOT IN ORACLE',
 ' Parental Guidance',
 ' Ages 10 & Up',
 ' Not in Oracle',
 ' MAX',
 ' Marvel Psr+',
 ' Ages 9+']

In [4]:
import numpy as np

df = df.astype(np.float32)

ValueError: ignored

In [5]:
n = 15000
df = df.iloc[:n]

In [6]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['Rating'].values.astype('U'))

In [7]:
cos_similarity = linear_kernel(tfidf_matrix, tfidf_matrix)

In [8]:
indices=pd.Series(df.index,index=df['comic_name'])
titles=df['comic_name']
def recommend_comics(title):
    idx = indices[title]
    similarity_scores = list(enumerate(cos_similarity[idx]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[1:11]
    data = []
    for i in similarity_scores:
        item = []
        temp_df = df[df['comic_name'] == indices.index[i[0]] ]
        item.extend(list(temp_df.drop_duplicates('comic_name')['comic_name'].values ))
        data.append(item)
    return data

In [9]:
recommend_comics("A Year of Marvels: August Infinite Comic (2016)")

[['A Year of Marvels: July Infinite Comic (2016)'],
 ['A Year of Marvels: June Infinite Comic (2016)'],
 ['A Year of Marvels: March Infinite Comic (2016)'],
 ['A Year of Marvels: May Infinite Comic (2016)'],
 ['A Year of Marvels: November Infinite Comic (2016)'],
 ['A Year of Marvels: October Infinite Comic (2016)'],
 ['A Year of Marvels: September Infinite Comic (2016)'],
 ['A Year of Marvels: The Amazing (2016)'],
 ['A Year of Marvels: The Incredible (2016)'],
 ['A+X (2012 - 2014)']]